# Eleuther AI Harness

This notebook can be used to run tasks of the eleuther AI harness on most types of models and upload the results to Zeno.


To run the harness, first clone and install its codebase.


In [ ]:
!git clone git@github.com:EleutherAI/lm-evaluation-harness.git
%pip install -e lm-evaluation-harness

In [ ]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
from lm_eval import evaluator, tasks, utils
import json
import os
from pathlib import Path
import numpy as np
import numbers

API_KEY = os.environ["ZENO_API_KEY"]

## Settings

The following settings can be configured to your needs.

**Warning: This might take very long to run depending on your hardware. We recommend setting the limit argument so as to only evaluate a subset of the task.**


In [ ]:
args = {
    "model": "hf-causal",  # for huggingface models
    "model_args": "pretrained=EleutherAI/gpt-neo-2.7B",  # this specifies the HF model to use
    "model_name": "llama-7b",  # used to identify the model on upload
    "tasks": "bigbench_geometric_shapes",  # tasks to run, multiple tasks separated by commas
    "num_fewshot": 3,  # num fewshot examples to include in the prompt
    "device": "cuda",  # device to run the model on, "cuda" if you have a GPU
    "output_path": "./output/latest_results.json",  # path to write results to
    "limit": 250,  # limit on the number of examples to run
    "no_cache": False,  # don't cache results
    "output_base_path": "./output",  # where to write the outputs to
}

## Running Evaluation

This evaluates the model on the selected tasks and write the results to the outputs folder. This step might take a while.


In [ ]:
task_names = utils.pattern_match(task_names.split(","), tasks.ALL_TASKS)
results = evaluator.simple_evaluate(
    model=args["model"],
    model_args=args["model_args"],
    tasks=task_names,
    num_fewshot=args["num_fewshot"],
    batch_size=None,
    max_batch_size=None,
    device=args["device"],
    no_cache=args["no_cache"],
    limit=args["limit"],
    description_dict={},
    decontamination_ngrams_path=None,
    check_integrity=None,
    write_out=True,
    output_base_path=Path(args["output_base_path"], "tasks"),
)

dumped = json.dumps(results, indent=2)

os.makedirs(args["output_base_path"], exist_ok=True)
with open(Path(args["output_base_path"], "latest_results.json"), "w") as f:
    f.write(dumped)

## Get all Output Files for the Tasks

Loads all task output files to be able to combine them and upload them to Zeno.


In [ ]:
# Get all output files for the tasks
dir_path = Path(args["output_base_path"], "tasks")
files = list(dir_path.glob("*"))
tasks_dict = [
    {
        "name": file.name.replace("_write_out_info.json", "").replace("bigbench_", ""),
        "file": file,
    }
    for file in files
]

## Helper Functions to Format the Data

These functions are used to get the data in the right shape to be uploaded to Zeno.


In [ ]:
def common_prefix(strings: list[str]):
    prefix: list[str] = []
    prefix_lines: list[list[str]] = list(map(lambda x: x.splitlines(), strings))
    index = 0
    while all(i[index] == prefix_lines[0][index] for i in prefix_lines):
        prefix.append(prefix_lines[0][index])
        index += 1
    common_lines = list(map(lambda x: x[index], prefix_lines))
    common_in_line = os.path.commonprefix(common_lines)
    prefix.append(common_in_line)
    common = "\n".join(prefix)
    common_in_line_answers: list[list[str]] = list(
        map(lambda x: [x.replace(common_in_line, "")], common_lines)
    )
    remaining_answers = list(map(lambda x: x[index + 1 :], prefix_lines))
    answers = list(map(list.__add__, common_in_line_answers, remaining_answers))
    answers = list(map(lambda x: "\n".join(x), answers))
    return {"prefix": common, "answers": answers}


def get_prefix_answers(data):
    prefix_answers = []
    keys = list(map(lambda x: x.keys(), data))
    prompt_keys = list(
        map(lambda x: list(filter(lambda y: y.startswith("prompt_"), x)), keys)
    )
    for element_index, element in enumerate(data):
        prompts = list(map(lambda x: element[x], prompt_keys[element_index]))
        prefix_answers.append(common_prefix(prompts))
    return prefix_answers


def generate_dataset(data, prefix_answers, task_name):
    ids = list(map(lambda x: task_name + str(x["doc_id"]), data))
    labels = list(map(lambda x: x["truth"], data))
    df = pd.DataFrame(
        {
            "id": ids,
            "data": list(
                map(
                    lambda x: x["prefix"]
                    + "\n"
                    + "\n".join(list(map(lambda y: "- " + y, x["answers"]))),
                    prefix_answers,
                )
            ),
            "task": task_name,
            "labels": labels,
        }
    )
    return df


def generate_system_df(data, prefix_answers, df, task_name):
    ids = list(map(lambda x: task_name + str(x["doc_id"]), data))
    keys = list(map(lambda x: x.keys(), data))
    logit_keys = list(
        map(lambda x: list(filter(lambda y: y.startswith("logit_"), x)), keys)
    )
    answers = []
    correct_list = []
    for element_index, element in enumerate(data):
        answer = ""
        correct = False
        logits = list(
            filter(
                lambda y: isinstance(y, numbers.Number),
                map(lambda x: element[x], logit_keys[element_index]),
            )
        )
        if "acc_norm" in keys[element_index]:
            norm_logits = logits / np.array(
                [float(len(i)) for i in prefix_answers[element_index]["answers"]]
            )
            answer = prefix_answers[element_index]["answers"][np.argmax(norm_logits)]
            correct = df["labels"][element_index].endswith(answer)
            answer = (
                answer
                + "\n\n"
                + "Raw Pred.: "
                + ", ".join(map(lambda y: str(round(y, 2)), logits))
                + "\n\n"
                + "Norm Pred.: "
                + ", ".join(map(lambda y: str(round(y, 2)), norm_logits))
            )
        else:
            answer = prefix_answers[element_index]["answers"][np.argmax(logits)]
            correct = df["labels"][element_index].endswith(answer)
            answer = (
                answer
                + "\n\n"
                + "Pred.: "
                + ", ".join(map(lambda y: str(round(y, 2)), logits))
            )
        answers.append(answer)
        correct_list.append(correct)
    system_df = pd.DataFrame({"id": ids, "output": answers, "correct": correct_list})
    return system_df


# Accumulate data for all tasks
for index, task in enumerate(tasks_dict):
    data = json.load(open(task["file"]))
    prefix_answers = get_prefix_answers(data)
    if index == 0:
        df = generate_dataset(data, prefix_answers, task["name"])
        system_df = generate_system_df(data, prefix_answers, df, task["name"])
    else:
        current_df = generate_dataset(data, prefix_answers, task["name"])
        df = pd.concat([df, current_df])
        system_df = pd.concat(
            [
                system_df,
                generate_system_df(data, prefix_answers, current_df, task["name"]),
            ]
        )

## Create the Zeno Project

This will load an existing project if you already have a project with the same name.


In [ ]:
client = ZenoClient(API_KEY, endpoint=ENDPOINT)

project = client.create_project(
    name="Eleuther Harness Project",
    view="text-classification",
    metrics=[ZenoMetric(name="accuracy", type="mean", columns=["correct"])],
)

## Upload the Dataset

Uploads the task data to Zeno.

**Only run this if you want to upload fresh data. This will delete all existing systems.**


In [ ]:
project.upload_dataset(df, id_column="id", data_column="data", label_column="labels")

## Upload the System

Uploads the model output to Zeno.


In [ ]:
project.upload_system(
    system_df,
    name=args["model_name"],
    id_column="id",
    output_column="output",
)